<a href="https://colab.research.google.com/github/NitayAmiel/HW_AI_Course/blob/main/HW6_AI_course_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up


In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
HOME = "/content/drive/MyDrive/testing123/drone_data"

# NOTICE: Adjust the path above accordingly to your files' location

%cd {HOME}

BIRD_PATH = '/content/drive/MyDrive/testing123/bird_data'


/content/drive/.shortcut-targets-by-id/1--E2a3ior8D_isWan7ePmd_NmsNcy6Uk/drone_data


# **Preproccessing the Data**

# augmentations functions

In [3]:
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import random

def random_flip(image):
    return ImageOps.mirror(image)

def color_jitter(image, brightness_factor=0.2, contrast_factor=0.2, saturation_factor=0.2):
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1 + random.uniform(-brightness_factor, brightness_factor))

    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1 + random.uniform(-contrast_factor, contrast_factor))

    enhancer = ImageEnhance.Color(image)
    image = enhancer.enhance(1 + random.uniform(-saturation_factor, saturation_factor))

    return image

def random_rotation(image, max_angle=45):
    angle = random.uniform(-max_angle, max_angle)
    return image.rotate(angle)

augmentations = [random_flip, color_jitter, random_rotation]

In [18]:
import matplotlib.pyplot as plt

def display_image(img, title):
  plt.subplot(1, 2, 2)
  plt.imshow(img)
  plt.title(title)
  plt.show()

first we will generate random images and drone images

In [24]:
import os
import random
from PIL import Image
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
from IPython.display import display

OTHER = 0
DRONE = 1
BIRD = 2

# Seperating between the images and the annotations
images = [image for image in os.listdir(HOME) if image[image.rfind(".")+1:] == "jpg"]
labels = [label for label in os.listdir(HOME) if label[label.rfind(".")+1:] == "txt"]

drone_images = []
other_images =[]

for index,label in enumerate(labels):
  if index == 10:
    break
  # Just for following the runtime (running this cell takes 2 minutes approx.)
  if index % 10 == 0:
    print(f' {index = }')

  # Reading the coordinates from the annotations, Remember: the labels are as YOLO V8 format.
  with open(label) as label_file:
    content = label_file.read()
    lst = content.split(" ")
    lst[4] = lst[4][:-1] #removing \n
    x_center, y_center, width, height = float(lst[1]), float(lst[2]),float(lst[3]),float(lst[4])

    # The coordinates list is in this order : x_left, y_top, x_right, y_down
    coordinates = [x_center - width/2, y_center - height/2 ,x_center + width/2, y_center + height/2]
    image_drone = Image.open(images[index])
    pixels_width, pixels_height = image_drone.size

    # Represnting the coordinates as number of pixels, and not as float in [0,1]
    for i in range(4):
      coordinates[i] = int(coordinates[i]*pixels_width) if i%2 == 0 else int(coordinates[i]*pixels_height)

    # Cropping , converting to gray, reshaping, and then saving the binding box with respect lebel( a drone)
    bnbox = image_drone.crop(tuple(coordinates))
    box = bnbox.resize((32,32))
    box = image.img_to_array(box, dtype = np.uint8)
    reshaped_img_array = np.reshape(box, (32,32,3))
    drone_images.append((reshaped_img_array, DRONE))

    # apllying augmentation and append
    changed_image = augmentations[random.randint(0,2)](bnbox)
    changed_image = changed_image.resize((32,32))
    changed_image = image.img_to_array(changed_image, dtype = np.uint8)
    reshaped_img_array = np.reshape(changed_image, (32,32,3))
    drone_images.append((reshaped_img_array, DRONE))


    # Creating a random 28*28 picture, converting it to gray, reshaping it, and then saving with respect lebel(not a drone)
    x_random = random.randint(0,pixels_width-32)
    y_random = random.randint(0,pixels_height-32)
    not_drone_image = image_drone.crop((x_random, y_random,x_random+32, y_random+32 ))
    not_drone_image = image.img_to_array(not_drone_image, dtype = np.uint8)
    not_drone_image = np.reshape(not_drone_image, (32,32,3))
    other_images.append((not_drone_image, OTHER))





 index = 0


extracting the bird images

In [25]:
%cd {BIRD_PATH}

/content/drive/.shortcut-targets-by-id/1YTpCrZXrygq52NizdbEjLGw_TI4tbGyy/bird_data


In [26]:
images = [image for image in os.listdir(BIRD_PATH) if image[image.rfind(".")+1:] == "jpg"]
labels = [label for label in os.listdir(BIRD_PATH) if label[label.rfind(".")+1:] == "txt"]

bird_images = []

for index,label in enumerate(labels):
  if index == 10:
    break
  # Just for following the runtime (running this cell takes 2 minutes approx.)
  if index % 10 == 0:
    print(f' {index = }')

  # Reading the coordinates from the annotations, Remember: the labels are as YOLO V8 format.
  with open(label) as label_file:
    content = label_file.read()
    lst = content.split(" ")
    lst[4] = lst[4][:-1] #removing \n
    x_center, y_center, width, height = float(lst[1]), float(lst[2]),float(lst[3]),float(lst[4])

    # The coordinates list is in this order : x_left, y_top, x_right, y_down
    coordinates = [x_center - width/2, y_center - height/2 ,x_center + width/2, y_center + height/2]
    image_bird = Image.open(images[index])
    pixels_width, pixels_height = image_bird.size

    # Represnting the coordinates as number of pixels, and not as float in [0,1]
    for i in range(4):
      coordinates[i] = int(coordinates[i]*pixels_width) if i%2 == 0 else int(coordinates[i]*pixels_height)

    # Cropping , converting to gray, reshaping, and then saving the binding box with respect lebel( a drone)
    bnbox = image_bird.crop(tuple(coordinates))
    box = bnbox.resize((32,32))
    box = image.img_to_array(box, dtype = np.uint8)
    reshaped_img_array = np.reshape(box, (32,32,3))
    bird_images.append((reshaped_img_array, DRONE))

    # apllying augmentation and append
    changed_image = augmentations[random.randint(0,2)](bnbox)
    changed_image = changed_image.resize((32,32))
    changed_image = image.img_to_array(changed_image, dtype = np.uint8)
    reshaped_img_array = np.reshape(changed_image, (32,32,3))
    bird_images.append((reshaped_img_array, BIRD))


    # Creating a random 28*28 picture, converting it to gray, reshaping it, and then saving with respect lebel(not a drone)
    x_random = random.randint(0,pixels_width-32)
    y_random = random.randint(0,pixels_height-32)
    not_bird_image = image_bird.crop((x_random, y_random,x_random+32, y_random+32 ))
    not_bird_image = image.img_to_array(not_bird_image, dtype = np.uint8)
    not_bird_image = np.reshape(not_bird_image, (32,32,3))
    other_images.append((not_bird_image, OTHER))



 index = 0


**Applying augmentations like random flip, color jitter, and rotation**

displaying some images from the data set

In [ ]:
import matplotlib.pyplot as plt

# Write here how many pictures you wish to display from every data set
number_of_images_to_display = 20

counter = 0
for img in drone_images:
  title = "original drone" if counter %2 == 0 else "drone after augmentation"
  display_image(img[0], title)
  counter += 1
  if counter > number_of_images_to_display:
    break


counter = 0
for img in bird_images:
  title = "original bird" if counter %2 == 0 else "bird after augmentation"
  display_image(img[0], title)
  counter += 1
  if counter > number_of_images_to_display:
    break


counter = 0
for img in other_images:
  display_image(img[0], "other")
  counter += 1
  if counter > number_of_images_to_display:
    break


# converting to TensorFlow Datasets


In [ ]:
import tensorflow as tf

# Adjust by your choice, to how to seperate the data (train and test)
split_ratio = 0.85

data_list = drone_images + other_images + bird_images
random.shuffle(data_list)
split_point = int(len(data_list)*split_ratio)
train_list = data_list[:split_point]
test_list = data_list[split_point:]


# Function to create a generator
def data_generator(list_of_data):
    for image_array, label in list_of_data:
        yield image_array, label

# Define the types and shapes of the elements in each tuple
element_spec = (
    tf.TensorSpec(shape=(32, 32, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
)

# Create the TensorFlow dataset
train_dataset = tf.data.Dataset.from_generator(lambda : data_generator(train_list), output_signature=element_spec)
test_dataset = tf.data.Dataset.from_generator(lambda : data_generator(test_list), output_signature=element_spec)


# Preprocess the data, it might take a while
train_images = np.array([img[0]  for img in train_dataset], dtype=np.float32)
test_images = np.array([img[0]  for img in test_dataset], dtype=np.float32)
train_labels = np.array([img[1]  for img in train_dataset], dtype=np.int32)
test_labels = np.array([img[1]  for img in test_dataset],dtype=np.int32)

# Converting the pixels to number in [0,1]
train_images = train_images.reshape((len(train_images), 32, 32, 3)).astype('float32') / 255
test_images = test_images.reshape((len(test_images), 32, 32, 3)).astype('float32') / 255


**size of data set**

 the data is perfectly balnced, 50% drones and 50% not


In [ ]:
print(len(data_list)) # the size of the entire data(including the random pictures)

8140


# creting a model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Create the CNN model
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))

# Convolutional layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolutional layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output and add fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Output layer with 3 neurons (3 categories)
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Display a summary of the model architecture, uneccessery - you can write it as a comment
model.summary()

# Defining the batch and the epochs
batch_size = 32
epochs = 10
# Train the model
model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size)


**evaluate the model**

In [ ]:
#Evaluate the model on a test dataset if available
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_accuracy}')

**example of prediction**

In [ ]:
import matplotlib.pyplot as plt
number_of_pictures_to_predict = 20

new_data = test_images[: number_of_pictures_to_predict]

predictions = model.predict(new_data)

# Convert predictions to binary (0 or 1) based on a threshold (e.g., 0.5), you can adjust for more or less surency
binary_predictions = (predictions > 0.5).astype(int)

counter = 0
# Plot the images along with their predictions
for i in range(len(new_data)):
    plt.imshow(new_data[i].reshape(28, 28), cmap='gray')
    real = "It is a drone" if test_labels[i] == DRONE else "It is NOT a drone"
    predict = "It is a drone" if binary_predictions[i][0] == DRONE else "It is NOT a drone"
    if real == predict:
      counter += 1
    plt.title(f'Prediction: {predict},  real Answer: {real}')
    plt.show()

print(f"In summary we have {counter} good predictions, out from {number_of_pictures_to_predict}")